In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D
from utils import *
from plot_utils import *
%matplotlib inline

In [ ]:
n_values = np.arange(5, 100, 1, dtype=int)

values_dict = {
    'star': dict(),
    'wheel': dict(),
    'comb': dict(),  
    'agave step=1': dict(),
    'agave step=n/3': dict(),
    'agave step=n': dict(),
    'complete_bipartite a=b': dict(),
    'complete_bipartite a=b/3': dict(),
    'complete': dict()
}

excluded_metrics = ['frac', 'm3', 'm4', 'W1', 'W2']


for n in tqdm(n_values):
    
    # Star graph (n nodes: center + n-1 leaves)
    G = nx.star_graph(n - 1)
    P = stochastic_matrix_calculator(G)
    append_values_plot_1(G, P, 'star', values_dict, excluded_metrics)
    
    # Wheel graph (n nodes)
    G = nx.wheel_graph(n)
    P = stochastic_matrix_calculator(G)
    append_values_plot_1(G, P, 'wheel', values_dict, excluded_metrics)
    
    # Line graph of wheel (only if small enough)
    G_prime = nx.wheel_graph(n)
    G = nx.line_graph(G_prime)
    if G.number_of_nodes() < 200:
        append_values_plot_1(G, P, 'line_wheel', values_dict, excluded_metrics)
    
    # Complete bipartite: a = b (as balanced as possible)
    a = math.floor(n / 2)
    b = math.ceil(n / 2)
    G = nx.complete_bipartite_graph(n1=a, n2=b)
    P = stochastic_matrix_calculator(G)
    append_values_plot_1(G, P, 'complete_bipartite a=b', values_dict, excluded_metrics)
    
    # Complete bipartite: a = b/3  → so a : b ≈ 1 : 3
    a = math.floor(n / 4)
    b = n - a
    # Ensure both partitions are non-empty
    if a >= 1 and b >= 1:
        G = nx.complete_bipartite_graph(n1=a, n2=b)
        P = stochastic_matrix_calculator(G)
        append_values_plot_1(G, P, 'complete_bipartite a=b/3', values_dict, excluded_metrics)
    
    # Agave graphs
    # Step = 1
    G = create_agave_graph(n=n, steps=1)
    P = stochastic_matrix_calculator(G)
    append_values_plot_1(G, P, 'agave step=1', values_dict, excluded_metrics)
    
    # Step = n/3
    steps = max(1, math.floor(n / 3))
    G = create_agave_graph(n=n, steps=steps)
    P = stochastic_matrix_calculator(G)
    append_values_plot_1(G, P, 'agave step=n/3', values_dict, excluded_metrics)
    
    # Step = n
    steps = n
    G = create_agave_graph(n=n, steps=steps)
    P = stochastic_matrix_calculator(G)
    append_values_plot_1(G, P, 'agave step=n', values_dict, excluded_metrics)
    
    # Complete graph
    G = nx.complete_graph(n)
    P = stochastic_matrix_calculator(G)
    append_values_plot_1(G, P, 'complete', values_dict, excluded_metrics)
    
    # Comb graph (≈ n nodes total)
    G = create_comb_graph(n)
    if G.number_of_nodes() >= 2:  # ensure it's a valid non-trivial graph
        P = stochastic_matrix_calculator(G)
        append_values_plot_1(G, P, 'comb', values_dict, excluded_metrics)

In [ ]:
values_df = pd.DataFrame(values_dict).T.reset_index(drop=False)
values_df.rename(columns={'index':'Graph_name'})

In [ ]:
values_df.to_csv("SavedNetworks/artificial_networks.csv")

In [ ]:
values_df = pd.read_csv("SavedNetworks/artificial_networks.csv")

In [ ]:
graph_keys = list(values_dict.keys())
exclude_keys = []

colors = ['red', 'blue', 'orange', 'purple', 'teal', 'slategrey', 'brown', 'violet', 'lime', 'black']

markers = ['o', 'v', '8', 'D', '^', '*', 'P', 'X', 'h', 'd']
marker_size = 7


legend_names = {
    'comb': 'Comb Graph',
    'star': 'Star Graph',
    'wheel': 'Wheel Graph',
    'complete_bipartite a=b': 'Complete Bipartite (Type1)',
    'complete_bipartite a=b/3': 'Complete Bipartite (Type2)',
    'complete': 'Complete Graph',
    'line_wheel': 'Line Wheel Graph',
    'agave step=1': 'Agave (Type1)',
    'agave step=n/3': 'Agave (Type2)',
    'agave step=n': 'Agave (Type3)'
}

plt.figure(figsize=(20, 12))
font = {'family': 'times new roman', 'weight': 'bold', 'size': 20}

for i, key in enumerate(graph_keys):
    if key in exclude_keys:
        continue
        

    if key == 'line_wheel':
        n_vals_all = np.array(values_dict[key]['n'])
        H_vals_all = np.array(values_dict[key]['H'])
        mask = n_vals_all < 100
        n_values = n_vals_all[mask]
        H_values = H_vals_all[mask]
    else:
        n_values = values_dict[key]['n']
        H_values = values_dict[key]['H']
    

    label = legend_names.get(key, key.replace('_', ' ').title() + " Graph")
    
    plt.plot(
        n_values,
        H_values, 
        marker=markers[i % len(markers)], 
        c=colors[i % len(colors)], 
        markeredgecolor='black',
        markerfacecolor=colors[i % len(colors)],
        markersize=marker_size,
        label=label,
        linewidth=3
        
    )


plt.legend(
    prop=font,
    ncol=3,                     
    bbox_to_anchor=(0.98, 0.055), 
    loc='lower right',       
    frameon=True,
    fancybox=True,
    shadow=False
)


plt.ylabel('Heterogeneity Index (H)', 
           family='times new roman', 
           weight='bold', 
           size=20,
           labelpad=25) 

plt.xlabel('Number of Nodes', 
           family='times new roman', 
           weight='bold', 
           size=25,
           labelpad=10)


plt.subplots_adjust(left=0.13, bottom=0.12, right=0.95, top=0.95)


plt.xticks(np.arange(0, 101, 10), family='times new roman', weight='bold', size=23)
plt.xlim(0, 100)
plt.yticks(np.arange(0, 0.51, 0.05),family='times new roman', weight='bold', size=23)
plt.grid(color='black', linestyle='--', linewidth=0.5)
plt.savefig("Figures/artificial_networks_heterogeneity_index.png", dpi=500)
plt.show()

In [ ]:
graph_keys = list(values_dict.keys())
exclude_keys = []

colors = ['red', 'blue', 'orange', 'purple', 'teal', 'slategrey', 'brown', 'violet', 'lime', 'black']

markers = ['o', 'v', '8', 'D', '^', '*', 'P', 'X', 'h', 'd']
marker_size = 7


legend_names = {
    'comb': 'Comb Graph',
    'star': 'Star Graph',
    'wheel': 'Wheel Graph',
    'complete_bipartite a=b': 'Complete Bipartite (Type1)',
    'complete_bipartite a=b/3': 'Complete Bipartite (Type2)',
    'complete': 'Complete Graph',
    'line_wheel': 'Line Wheel Graph',
    'agave step=1': 'Agave (Type1)',
    'agave step=n/3': 'Agave (Type2)',
    'agave step=n': 'Agave (Type3)'
}

plt.figure(figsize=(20, 12))
font = {'family': 'times new roman', 'weight': 'bold', 'size': 17}

for i, key in enumerate(graph_keys):
    if key in exclude_keys:
        continue
        

    if key == 'line_wheel':
        m2_vals_all = np.array(values_dict[key]['m2'])
        H_vals_all = np.array(values_dict[key]['H'])
        mask = m2_vals_all < 100
        m2_values = m2_vals_all[mask]
        H_values = H_vals_all[mask]
    else:
        m2_values = values_dict[key]['m2']
        H_values = values_dict[key]['H']
    

    label = legend_names.get(key, key.replace('_', ' ').title() + " Graph")
    
    plt.plot(
        m2_values,
        H_values, 
        marker=markers[i % len(markers)], 
        c=colors[i % len(colors)], 
        markeredgecolor='black',
        markerfacecolor=colors[i % len(colors)],
        markersize=marker_size,
        label=label,
        linewidth=3
        
    )

plt.xlabel('m2', family='times new roman', weight='bold', size=20)
plt.ylabel('Heterogeneity Index (H)', family='times new roman', weight='bold', size=20)

plt.legend(
    prop=font,
    ncol=3,                     
    bbox_to_anchor=(0.98, 0.07), 
    loc='lower right',       
    frameon=True,
    fancybox=True,
    shadow=False
)


plt.ylabel('Heterogeneity Index (H)', 
           family='times new roman', 
           weight='bold', 
           size=20,
           labelpad=20) 

plt.xlabel('Number of Nodes', 
           family='times new roman', 
           weight='bold', 
           size=20,
           labelpad=10)


plt.subplots_adjust(left=0.13, bottom=0.12, right=0.95, top=0.95)


# plt.xticks(np.arange(0, 101, 10), family='times new roman', weight='bold', size=20)
# plt.xlim(0, 100)
# plt.yticks(np.arange(0, 0.51, 0.05),family='times new roman', weight='bold', size=20)
plt.grid(color='black', linestyle='--', linewidth=0.5)
# plt.savefig("Figures/artificial_networks_heterogeneity_index.png", dpi=1000, bbox_inches='tight')
plt.show()

In [ ]:
graph_keys = list(values_dict.keys())
exclude_keys = []

colors = ['red', 'blue', 'orange', 'purple', 'teal', 'slategrey', 'brown', 'violet', 'lime', 'black']
markers = ['o', 'v', '8', 'D', '^', '*', 'P', 'X', 'h', 'd']

legend_names = {
    'comb': 'Comb Graph',
    'star': 'Star Graph',
    'wheel': 'Wheel Graph',
    'complete_bipartite a=b': 'Complete Bipartite (Type1)',
    'complete_bipartite a=b/3': 'Complete Bipartite (Type2)',
    'complete': 'Complete Graph',
    'line_wheel': 'Line Wheel Graph',
    'agave step=1': 'Agave (Type1)',
    'agave step=n/3': 'Agave (Type2)',
    'agave step=n': 'Agave (Type3)'
}

fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(111, projection='3d')

font = {'family': 'times new roman', 'weight': 'bold', 'size': 20}

for i, key in enumerate(graph_keys):
    if key in exclude_keys:
        continue

    if key == 'line_wheel':
        H_vals = np.array(values_dict[key]['H'])
        m2_vals = np.array(values_dict[key]['m2'])
        n_vals = np.array(values_dict[key]['n'])
        mask = n_vals < 100
        H_vals, m2_vals, n_vals = H_vals[mask], m2_vals[mask], n_vals[mask]
    else:
        H_vals = np.array(values_dict[key]['H'])
        m2_vals = np.array(values_dict[key]['m2'])
        n_vals = np.array(values_dict[key]['n'])

    if len(H_vals) == 0:
        continue

    label = legend_names.get(key, key.replace('_', ' ').title() + " Graph")
    color = colors[i % len(colors)]
    marker = markers[i % len(markers)]

    ax.scatter(
        H_vals,                 # x: H
        m2_vals,                # y: m2
        n_vals,                 # z: number of nodes
        c=color,               
        marker=marker,
        s=80,                   
        edgecolors='black',
        linewidth=0.6,
        alpha=0.9,
        label=label
    )

ax.set_xlabel('Heterogeneity Index (H)', family='times new roman', weight='bold', size=25, labelpad=15)
ax.set_ylabel('m2', family='times new roman', weight='bold', size=25, labelpad=15)
ax.set_zlabel('Number of Nodes', family='times new roman', weight='bold', size=25, labelpad=20)

ax.set_xlim(0.5, 0)


ax.tick_params(axis='x')
ax.tick_params(axis='y')
ax.tick_params(axis='z')



for axis in [ax.xaxis, ax.yaxis, ax.zaxis]:
    axis.set_tick_params(labelsize=16)


ax.legend(
    prop=font,
    ncol=2,
    bbox_to_anchor=(0.29, 0.82),
    loc='upper left',
    title_fontsize=25,
    fontsize=25,
)

ax.grid(True, linestyle='--', linewidth=0.5, color='gray', alpha=0.6)
ax.view_init(elev=20, azim=-60)

plt.tight_layout()
plt.savefig("Figures/artificial_networks_3D.png", dpi=500)
plt.show()